In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col,when
# Azure storage detail
storage_account_name = "dnyaneshb"
storage_account_key = "3HYaZZYodJrg+x1bfXjm/GHj/JZ1u6DeLkSHVRgClPeHkrx1Qe4ljG9O62OId5E/GK6gcD9INax/+AStg9afyg=="
container_name = "feedgrain"

# Connecting spark with necessary configuration
spark = SparkSession.builder \
    .appName("AzureCSV") \
    .config(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key) \
    .getOrCreate()

# file path of Azure storage
read_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/Feedgrain/FeedGrains.csv"

# Read the file from azure storage
df = spark.read.csv(
    read_path,
    
    header=True,
    inferSchema=True
)

df.show()

+-----------+--------------+-----------------+-------------------+---------------+---------+-------------------------+---------------+-----------------+---------------+-----------------+----------+-------------+-------+---------------+-----------------+-------------+--------------------+------+
|SC_Group_ID| SC_Group_Desc|SC_GroupCommod_ID|SC_GroupCommod_Desc|SC_Geography_ID|SortOrder|SC_GeographyIndented_Desc|SC_Commodity_ID|SC_Commodity_Desc|SC_Attribute_ID|SC_Attribute_Desc|SC_Unit_ID| SC_Unit_Desc|Year_ID|SC_Frequency_ID|SC_Frequency_Desc|Timeperiod_ID|     Timeperiod_Desc|Amount|
+-----------+--------------+-----------------+-------------------+---------------+---------+-------------------------+---------------+-----------------+---------------+-----------------+----------+-------------+-------+---------------+-----------------+-------------+--------------------+------+
|          2|Supply and use|                9|             Barley|              1|      0.8|            United S

In [2]:
df.printSchema()

root
 |-- SC_Group_ID: integer (nullable = true)
 |-- SC_Group_Desc: string (nullable = true)
 |-- SC_GroupCommod_ID: integer (nullable = true)
 |-- SC_GroupCommod_Desc: string (nullable = true)
 |-- SC_Geography_ID: integer (nullable = true)
 |-- SortOrder: double (nullable = true)
 |-- SC_GeographyIndented_Desc: string (nullable = true)
 |-- SC_Commodity_ID: integer (nullable = true)
 |-- SC_Commodity_Desc: string (nullable = true)
 |-- SC_Attribute_ID: integer (nullable = true)
 |-- SC_Attribute_Desc: string (nullable = true)
 |-- SC_Unit_ID: integer (nullable = true)
 |-- SC_Unit_Desc: string (nullable = true)
 |-- Year_ID: integer (nullable = true)
 |-- SC_Frequency_ID: integer (nullable = true)
 |-- SC_Frequency_Desc: string (nullable = true)
 |-- Timeperiod_ID: integer (nullable = true)
 |-- Timeperiod_Desc: string (nullable = true)
 |-- Amount: double (nullable = true)



In [3]:
from pyspark.sql.functions import trim
for c in df.columns:
    if df.schema[c].dataType.simpleString() == 'string':
        df = df.withColumn(c, trim(col(c)))


In [4]:
df = df.dropDuplicates()

In [5]:
df = df.dropna(how='any')

In [6]:
from pyspark.sql.types import DoubleType, IntegerType

df = df.withColumn("Amount", col("Amount").cast(DoubleType()))
df = df.withColumn("Year_ID", col("Year_ID").cast(IntegerType()))

In [7]:
from pyspark.sql.functions import col, expr

# To make Amount column is cast to double
df = df.withColumn("Amount", col("Amount").cast("double"))

# Calculate Q1 and Q3
quantiles = df.approxQuantile("Amount", [0.25, 0.75], 0.01)
q1, q3 = quantiles[0], quantiles[1]
iqr = q3 - q1

# Define bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter outliers
outliers = df.filter((col("Amount") < lower_bound) | (col("Amount") > upper_bound))



In [8]:
# rows count after preprocessing
final_count = df.count()
print(f"Rows After Cleaning: {final_count}")

Rows After Cleaning: 515749


In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("WriteLargeDataToPostgres") \
    .config("spark.jars", "C:/Users/dnyan/SPARK/spark-3.5.5-bin-hadoop3/spark-3.5.5-bin-hadoop3/jars/postgresql-42.6.0.jar") \
    .getOrCreate()

# to save the data in postgresql using jdbc
df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5433/postgres") \
    .option("dbtable", "Feedgrain") \
    .option("user", "postgres") \
    .option("password", "8192") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
.save()

In [10]:
#How can historical trends  be leveraged to accurately forecast future feed grain prices using machine learning models?

In [11]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
targetgrains = ["Oats", "Corn", "Barley", "Sorghum"]
filtered = df.filter(
    (df["SC_Commodity_Desc"].isin(targetgrains)) &
    (df["SC_Frequency_Desc"] == "Annual") &
    (df["SC_GeographyIndented_Desc"] == "United States") &
    (df["SC_Attribute_Desc"].contains("Price"))
).select("Year_ID", "SC_Commodity_Desc", "Amount").dropna()

In [13]:
df = filtered.toPandas()# convert filter data to pandas because it does not xgboost in pyspark 
#without xgboost4j-spark_2.12:1.6.1 which is not compatable with my spark version. so i use allternative to convert it into pandas frames

In [14]:
dfrows = []

for _, row in df.iterrows():
    for month in range(1, 13):
        dfrows.append({
            "Year_ID": row["Year_ID"],
            "Month_ID": month,
            "SC_Commodity_Desc": row["SC_Commodity_Desc"],
            "Amount": row["Amount"]
        })

expanded_df = pd.DataFrame(dfrows)

In [15]:
expanded_df["commodity_index"] = expanded_df["SC_Commodity_Desc"].astype("category").cat.codes

# this add seasonal features for month
expanded_df["sin_month"] = np.sin(2 * np.pi * expanded_df["Month_ID"] / 12).round(2)
expanded_df["cos_month"] = np.cos(2 * np.pi * expanded_df["Month_ID"] / 12).round(2)


In [16]:
# Here trainig the xgboost model
X = expanded_df[["Year_ID", "commodity_index", "sin_month", "cos_month"]]
y = expanded_df["Amount"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=100, objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [17]:
y_pred = model.predict(X_test)

In [18]:
# Evalution metrics for xgboost model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_pred) * 100
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
r2 = r2_score(y_test, y_pred)

# printing the Output for metrics

print(f" RMSE : {rmse:.2f}")
print(f" MSE  : {mse:.2f}")
print(f" MAPE : {mape:.2f}%")
print(f" R²    : {r2:.2f}")


 RMSE : 0.81
 MSE  : 0.66
 MAPE : 14.55%
 R²    : 0.81


In [19]:
startyear = datetime.now().year + 1
endyear = startyear  # future price for next Only one year


In [20]:
future_dates = pd.date_range(start=f"{startyear}-01-01", end=f"{startyear}-12-01", freq="MS")

forecastdata = []
for grain in targetgrains:
    grain_index = expanded_df[expanded_df["SC_Commodity_Desc"] == grain]["commodity_index"].iloc[0]
    for date in future_dates:
        forecastdata.append({
            "Year_ID": date.year,
            "Month_ID": date.month,
            "SC_Commodity_Desc": grain,
            "commodity_index": grain_index,
            "sin_month": np.round(np.sin(2 * np.pi * date.month / 12), 2),
            "cos_month": np.round(np.cos(2 * np.pi * date.month / 12), 2)
        })

future_df = pd.DataFrame(forecastdata)

In [21]:
X_future = future_df[["Year_ID", "commodity_index", "sin_month", "cos_month"]]
future_df["Predicted_Price"] = model.predict(X_future)

In [22]:

print(future_df.head(5))

   Year_ID  Month_ID SC_Commodity_Desc  commodity_index  sin_month  cos_month  \
0     2026         1              Oats                2       0.50       0.87   
1     2026         2              Oats                2       0.87       0.50   
2     2026         3              Oats                2       1.00       0.00   
3     2026         4              Oats                2       0.87      -0.50   
4     2026         5              Oats                2       0.50      -0.87   

   Predicted_Price  
0         3.414608  
1         3.390695  
2         3.383761  
3         3.401944  
4         3.417218  


In [23]:
from sqlalchemy import create_engine

db_user = 'postgres'
db_password = '8192'
db_host = 'localhost'         
db_port = '5433'           
db_name = 'postgres'

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
future_df.to_sql("xgboost", engine, if_exists="replace", index=False)
print("Forecasts saved to PostgreSQL!")

Forecasts saved to PostgreSQL!


In [24]:
# 2 How has the market demand for major feed grains evolved in relation to their production over the years?

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, sum as spark_sum
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
from sqlalchemy import create_engine
from pyspark.sql.functions import sum as spark_sum

In [26]:
from pyspark.sql import SparkSession
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5433/postgres") \
    .option("dbtable", "Feedgrain") \
    .option("user", "postgres") \
    .option("password", "8192") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [27]:
from pyspark.sql.functions import col

feed_grains = ["Corn", "Barley", "Oats", "Sorghum"]
filtered_df = df.filter(col("SC_Commodity_Desc").isin(feed_grains))

In [28]:
df_cat = filtered_df.withColumn(
    "Category",
    when(col("SC_Attribute_Desc").contains("Production"), "Production")
    .when(col("SC_Attribute_Desc").rlike("Use|Utilization|Export|Feed|Residual|Food|Seed|Industrial"), "Demand")
)

In [29]:
# Filter relevant records
df_classified = df_cat.filter(col("Category").isNotNull())

In [30]:
df_grouped = df_classified.groupBy("Year_ID", "SC_Commodity_Desc", "Category") \
    .agg(spark_sum("Amount").alias("Total")) \
    .groupBy("Year_ID", "SC_Commodity_Desc") \
    .pivot("Category").sum("Total") \
    .dropna()

In [31]:
indexer = StringIndexer(inputCol="SC_Commodity_Desc", outputCol="GrainIndex")
df_indexed = indexer.fit(df_grouped).transform(df_grouped)

In [32]:
assembler = VectorAssembler(inputCols=["Year_ID", "Production", "GrainIndex"], outputCol="features")
assembled = assembler.transform(df_indexed)


In [33]:
train, test = assembled.randomSplit([0.8, 0.2], seed=42)

# 10. Train the model
rf = RandomForestRegressor(featuresCol="features", labelCol="Demand", numTrees=100)
model = rf.fit(train)

In [34]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(labelCol="Demand", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(f"R² for combined model: {r2:.2f}")

R² for combined model: 0.96


In [35]:
Demand = predictions.select("Year_ID", "SC_Commodity_Desc", "Production", "Demand", "prediction") \
                               .withColumnRenamed("prediction", "Predicted_Demand")

In [ ]:
print(Demand.head(5))

+-------+-----------------+----------+--------+--------------------+
|Year_ID|SC_Commodity_Desc|Production|  Demand|    Predicted_Demand|
+-------+-----------------+----------+--------+--------------------+
|   1960|           Barley| 69994.005| 76301.0|   5763901.759541265|
|   1961|             Corn|119994.803|140448.0| 2.082674108638114E8|
|   1962|           Barley| 78256.726| 81734.0|   5767623.265941646|
|   1963|          Sorghum| 29435.394| 32724.0|3.0402048625658005E7|
|   1965|          Sorghum| 29957.698| 36805.0|3.0403280619875666E7|
+-------+-----------------+----------+--------+--------------------+
only showing top 5 rows



In [37]:
from sqlalchemy import create_engine

Demand.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{db_host}:{db_port}/{db_name}") \
    .option("dbtable", "demand") \
    .option("user", db_user) \
    .option("password", db_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("Forecasts saved to PostgreSQL!")

Forecasts saved to PostgreSQL!


In [38]:
#What are the patterns and trends in feed grain production over different seasons or years?

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, count
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
import os

In [40]:
# Fix for Windows Python path
os.environ["PYSPARK_PYTHON"] = "python"


#  Filter for only Production 
df = df.filter(
    (col("SC_Attribute_Desc") == "Production") &
    (col("SC_GeographyIndented_Desc") != "World less U.S.") &
    (col("SC_GroupCommod_Desc").isin("Oats", "Corn", "Sorghum", "Barley"))
)

#  Drop rows with missing key values
df = df.dropna(subset=[
    "SC_GroupCommod_Desc", "SC_GeographyIndented_Desc", 
    "Year_ID", "Timeperiod_Desc", "Amount"
])

In [41]:
# Compute quantile thresholds for categorizing production
quantiles = df.approxQuantile("Amount", [0.33, 0.66], 0.01)
low_thresh, high_thresh = quantiles

#  Add Production_Level column
df = df.withColumn(
    "Production_Level",
    when(col("Amount") <= low_thresh, "Low")
    .when(col("Amount") <= high_thresh, "Medium")
    .otherwise("High")
)

In [42]:
# Feature preparation
indexers = [
    StringIndexer(inputCol="SC_GroupCommod_Desc", outputCol="GrainIndex"),
    StringIndexer(inputCol="SC_GeographyIndented_Desc", outputCol="RegionIndex"),
    StringIndexer(inputCol="Timeperiod_Desc", outputCol="SeasonIndex"),
    StringIndexer(inputCol="Production_Level", outputCol="label")
]

assembler = VectorAssembler(
    inputCols=["GrainIndex", "RegionIndex", "SeasonIndex", "Year_ID"],
    outputCol="features"
)

In [43]:
#  Random Forest Classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)

# Step 9: Pipeline
pipeline = Pipeline(stages=indexers + [assembler, rf])

# Step 10: Split data
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Step 11: Fit model
model = pipeline.fit(train_data)

# Step 12: Predict
predictions = model.transform(test_data)


In [44]:


evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = f1_evaluator.evaluate(predictions)

# Print metrics
print(f"\n Model Accuracy: {accuracy:.2%}")
print(f" F1 Score: {f1_score:.2%}")

#  Grouped analysis by Year, Grain, Region
agg_df = df.groupBy("Year_ID", "SC_GroupCommod_Desc", "SC_GeographyIndented_Desc").agg(
    avg("Amount").alias("Avg_Production"),
    count(when(col("Production_Level") == "High", True)).alias("High_Count"),
    count(when(col("Production_Level") == "Low", True)).alias("Low_Count")
)



 Model Accuracy: 91.95%
 F1 Score: 92.04%


In [45]:
# Rename columns for output
agg_df = agg_df.withColumnRenamed("Year_ID", "Year") \
               .withColumnRenamed("SC_GroupCommod_Desc", "Grain") \
               .withColumnRenamed("SC_GeographyIndented_Desc", "Region")

# Optional: Show aggregated results
agg_df.show()


+----+-------+-------------+--------------+----------+---------+
|Year|  Grain|       Region|Avg_Production|High_Count|Low_Count|
+----+-------+-------------+--------------+----------+---------+
|1874|   Oats|United States|       272.501|         0|        1|
|2008|   Oats|United States|        90.051|         0|        1|
|1967|   Corn|United States|      4860.372|         1|        0|
|1877|   Oats|United States|        435.33|         0|        0|
|2007|   Corn|United States|     13037.875|         1|        0|
|1888|   Corn|United States|      2250.632|         1|        0|
|2020| Barley|United States|        170.71|         0|        1|
|1993|Sorghum|United States|       534.172|         0|        0|
|2005| Barley|United States|       211.896|         0|        1|
|1932|   Oats|United States|      1254.584|         1|        0|
|1903| Barley|United States|       149.335|         0|        1|
|1988|Sorghum|United States|       576.686|         0|        0|
|1942| Barley|United Stat

In [46]:
from pyspark.sql import SparkSession

# Create Spark Session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("ReadDataFromPostgres") \
    .config("spark.jars", "C:/Users/dnyan/SPARK/spark-3.5.5-bin-hadoop3/spark-3.5.5-bin-hadoop3/jars/postgresql-42.6.0.jar") \
    .getOrCreate()
agg_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5433/postgres") \
    .option("dbtable", "Trends") \
    .option("user", "postgres") \
    .option("password", "8192") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()